# Conversion to ONNX
ONNX is a different format for running machine learning models. The ONNX format is much faster on CPU, sometimes 5 times as fast as PyTorch!

While the EAWSW model is designed to be small, accurate and accessible, for some people it's still too much to run...

Hosting the model as a free service for players is an option. An ONNX version of the model allows us to host the model on CPU yet have faster response times! Given that the model is made in a time with chip shortage, running on hardware I already have inside a server is efficient, scalable and cheaper.

An important note is that ONNX doesn't execute logic by itself, and you have to do that yourself, `onnx_model_manager.py` intends to deal with this for us.

In [1]:
%load_ext autoreload
%autoreload 2

from model_utils import train_model, split_data, split_branches, get_model, set_pretrained_model_dropout, get_dataset
from config import Config
import json
import matplotlib.pyplot as plt
%matplotlib inline
import math
import random
import time
import onnx
import logging
from onnx_model_manager import OnnxModelManager
from onnxruntime.quantization import quantize_dynamic, QuantType
import os
import datasets
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from model_manager import ModelManager

In [2]:
saved_model_path = os.path.join("models", "awsw_main")
saved_model_onnx_path = os.path.join("models", "awsw_onnx")
if not os.path.exists(os.path.join(saved_model_path, "special_tokens_map.json")):
    print("Copying config files from huggingface (needed for conversion)... WARNING: this assumes the structure of the model isn't changed!")
    !cd $saved_model_path && git clone https://huggingface.co/EleutherAI/gpt-neo-125M
    !cp -n $saved_model_path/gpt-neo-125M/* $saved_model_path
    !rm -rf $saved_model_path/gpt-neo-125M
if not os.path.exists(os.path.join(saved_model_onnx_path, "model.onnx")):
    !python3 -m transformers.onnx --model=$saved_model_path --feature=causal-lm-with-past $saved_model_onnx_path

Copying config files from huggingface (needed for conversion)... WARNING: this assumes the structure of the model isn't changed!
Cloning into 'gpt-neo-125M'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 44 (delta 20), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (44/44), 543.14 KiB | 1.27 MiB/s, done.
Using framework PyTorch: 1.10.1+cu113
Overriding 1 configuration item(s)
	- use_cache -> True
/usr/local/lib/python3.8/dist-packages/transformers/models/gpt_neo/modeling_gpt_neo.py:555: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert batch_size > 0, "batch_size has to be defined and > 0"
Validating ONNX model...
	-[✓] ONNX model output names match ref

In [ ]:
def optimize_onnx():
    model_quant = os.path.join(saved_model_onnx_path, "model_quant.onnx")
    if not os.path.exists(model_quant):
        model_fp32 = os.path.join(saved_model_onnx_path, "model.onnx")
        model_opt = os.path.join(saved_model_onnx_path, "model-opt.onnx")
        quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type = QuantType.QInt8)
        #!rm $model_opt
optimize_onnx()

Ignore MatMul due to non constant B: /[MatMul_111]
Ignore MatMul due to non constant B: /[MatMul_142]
Ignore MatMul due to non constant B: /[MatMul_246]
Ignore MatMul due to non constant B: /[MatMul_277]
Ignore MatMul due to non constant B: /[MatMul_381]
Ignore MatMul due to non constant B: /[MatMul_412]
Ignore MatMul due to non constant B: /[MatMul_516]
Ignore MatMul due to non constant B: /[MatMul_547]
Ignore MatMul due to non constant B: /[MatMul_651]
Ignore MatMul due to non constant B: /[MatMul_682]
Ignore MatMul due to non constant B: /[MatMul_786]
Ignore MatMul due to non constant B: /[MatMul_817]
Ignore MatMul due to non constant B: /[MatMul_921]
Ignore MatMul due to non constant B: /[MatMul_952]
Ignore MatMul due to non constant B: /[MatMul_1056]
Ignore MatMul due to non constant B: /[MatMul_1087]
Ignore MatMul due to non constant B: /[MatMul_1191]
Ignore MatMul due to non constant B: /[MatMul_1222]
Ignore MatMul due to non constant B: /[MatMul_1326]
Ignore MatMul due to non c

In [4]:
# Tell pytorch to run this model on the GPU.
device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
# device_name = 'cpu'
device = torch.device(device_name)

onnx_model_manager = OnnxModelManager(os.path.join(saved_model_onnx_path, "model-opt.onnx"))
onnx_model_manager_quant = OnnxModelManager(os.path.join(saved_model_onnx_path, "model_quant.onnx"))
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125M')
model = AutoModelForCausalLM.from_pretrained(saved_model_path)
model.to(device)
model.eval()
model_manager = ModelManager(model=model, tokenizer=tokenizer, device=device)
print(f"Pretrained model loaded on {device_name}")

Pretrained model loaded on cpu


In [5]:
prompt = "In my dreams, I'm a dragon"
for i in range(2):
    print("ONNX:", onnx_model_manager.say_raw(prompt, do_sample=True))
    print("ONNX (Quantized):", onnx_model_manager_quant.say_raw(prompt, do_sample=True))
    print("PyTorch:", model_manager.say_raw(prompt, 50, 0.7))
    print('-' * 100)

ONNX: In my dreams, I'm a dragon. I'm not even old enough to remember anything."<d><scn>black<msg>Lo "I see."#mvn
<p>title^1< imaginary number]{ for example}I'm not. You know what? This is a completely different story. You won't find out if you will by next year"#mv."<d><scn>np2x<msg>Ry "I don't know, Reza."<p><msgary>c "What do you think of Lorem?"><d><scn>np1x<
ONNX (Quantized): In my dreams, I'm a dragon at heart."<d> <p> byremyreid<msg>or "Meet with [a city-based entity]" <d>("Be that to me?"<|endoftext|>
PyTorch: In my dreams, I'm a dragon."<p><msg>c "Hey Maverick! Over here!"<d><scn>o2<msg>Mv "Hey [player_name]! How are you?"<p><msg>c "Not at all."<d><scn>o2<msg>Br "Hey, I'm Daniel. 'Til the task at hand is pretty simple."<|endoftext|>
----------------------------------------------------------------------------------------------------
ONNX: In my dreams, I'm a dragon."<|endoftext|>
ONNX (Quantized): In my dreams, I'm a dragon and a little girl from a previous life. I grew up arou

# Testing

We created a few past (for context) + present prompts (player input) and see the different reactions. This way, we can test the models across different iterations.
The first test involves a old prompt to compare the pre-trained model with the one trained on AWSW. Did it manage to store it's data well? Is it able to write down things that have nothing to do with AWSW? (So we know we didn't overfit).

**This test generates boring and repetetive** replies! It's because we use no good sampling algorithm, but it does give us a indication of what the model has learned!

In [ ]:
prompts = [
    ('<p><msg>c "Hey Remy!"<d><scn>park2<msg>Ry "Hello, [player_name]."', "How are you?"),
    ('<p><msg>c "I was with Lorem today."<d><scn>park2<msg>Ad "Very nice."', "What do you think of Lorem?"),
    ('<p><msg>m "In Tatsu park, Adine and I sat down."', "Oh my god, Adine. What is this?"),
    ('<p><msg>m "I sat down on a chair in Anna\'s lab."', "What will we do here?"),
]

for (past, prompt) in prompts:
    print(f"Prompt: {prompt}")
    reply = model_manager.say(past, prompt)
    print(f"[Pytorch] Reply: {reply}\n")
    reply = onnx_model_manager.say(past, prompt)
    print(f"[ONNX] Reply: {reply}\n")
    reply = onnx_model_manager_quant.say(past, prompt)
    print(f"[ONNX Quantized] Reply: {reply}\n")
    print("-" * 10)

Prompt: How are you?
[Pytorch] Reply: park2<msg>Ry "I was and are, [player_name]."<p><msg>c "Hello Zhong"<d><scn>park2<msg>Zh "Hey [player_name]!"<p><msg>c "I see."<d><scn>park2<msg>Ry "I

[ONNX] Reply: park2<msg>Ry "I was and are, [player_name]."<p><msg>c "Hello Zhong"<d><scn>park2<msg>Zh "Hey [player_name]!"<p><msg>c "I see."<d><scn>park2<msg>Ry "I'm not sure if I can be of much help."<p><msg>c "go to remy"<d><scn>remyapt<msg>Ry "Oh, [player_name], I wasn't expecting visitors

[ONNX Quantized] Reply: <msg>c "Then, maybe I should put that off for a moment. I might even see the end."<d><scn>park3<msg>Mv "Oh, I'm not sure that will do."<p><msg>c "And I'll never get over it, I'll be the only one. I'll probably die of it soon."<d><scn>park3<msg>Mv "If you can get to the end, I can finish it."<p><msg>c "And I'm not sure if that will be the outcome."

----------
Prompt: What do you think of Lorem?
[Pytorch] Reply: park2<msg>Ad "I think he's funny."<p><msg>c "I'm not sure if this is much use

# Sampling test

This is gonna be interesting!

In [ ]:
for (past, prompt) in prompts:
    print(f"Prompt: {prompt}")
    reply = model_manager.say(past, prompt, top_k = 50, top_p = 0.7)
    print(f"[Pytorch] Reply: {reply}\n")
    reply = onnx_model_manager.say(past, prompt, do_sample = True)
    print(f"[ONNX] Reply: {reply}\n")
    reply = onnx_model_manager_quant.say(past, prompt, do_sample = True)
    print(f"[ONNX Quantized] Reply: {reply}\n")
    print("-" * 10)

# RP test
Testing out the injected roleplay actions

In [ ]:
test_rps = [
    "Visit Lorem",
    "Meet with Lorem",
    "Visit Adine",
    "Fight",
    "Attack"
]

for rp in test_rps:
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    print("-" * 10)
    
print("Lowercase test")

for rp in test_rps:
    rp = rp[0].lower() + rp[1:]
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    rp = rp.lower()
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    print("-" * 10)